In [ ]:
import pandas as pd



In [ ]:
df=messages = pd.read_csv('SMSSpamCollection.txt', sep='\t',
                           names=["label", "message"])
df.head()


FileNotFoundError: [Errno 2] No such file or directory: 'SMSSpamCollection.txt'

In [ ]:
df.shape


In [ ]:
X=list(df['message'])


NameError: name 'df' is not defined

In [ ]:
y=list(df['label'])


NameError: name 'df' is not defined

In [ ]:
y

In [ ]:
y= list(pd.get_dummies(y,drop_first=True)['spam'])

NameError: name 'y' is not defined

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)

In [ ]:
from transformers import DistilBertTokenizerFast
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

In [ ]:
train_encodings = tokenizer(X_train, truncation=True, padding=True)
test_encodings = tokenizer(X_test, truncation=True, padding=True)

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split, RandomizedSearchCV, cross_val_score
from sklearn.ensemble import StackingClassifier, RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score, confusion_matrix, roc_curve, auc
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectFromModel
from xgboost import XGBClassifier
import matplotlib.pyplot as plt
import seaborn as sns
import shap
from tpot import TPOTClassifier

data = pd.read_csv('C:\\Users\\HP\\Downloads\\Code\\dna.csv')

sns.countplot(data['class'])
plt.title('Class Distribution')
plt.show()

X = data.drop(columns=['class'])
y = data['class']


poly = PolynomialFeatures(degree=2, interaction_only=True) # polynomial features
X_poly = poly.fit_transform(X)

selector = SelectFromModel(RandomForestClassifier(n_estimators=100))
X_selected = selector.fit_transform(X_poly, y)

X_train, X_test, y_train, y_test = train_test_split(X_selected, y, test_size=0.2, random_state=42)

base_learners = [
    ('rf', RandomForestClassifier(n_estimators=100)),
    ('gb', GradientBoostingClassifier(n_estimators=100)),
    ('xgb', XGBClassifier(n_estimators=100))
]


stacking_clf = StackingClassifier(
    estimators=base_learners,         # stacking classifier
    final_estimator=LogisticRegression(),
    cv=5
)

# pipeline
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('classifier', stacking_clf)
])

param_grid = {
    'classifier__final_estimator__C': [0.1, 1, 10],
    'classifier__rf__n_estimators': [50, 100, 200],
    'classifier__gb__learning_rate': [0.01, 0.1, 0.2],
    'classifier__xgb__learning_rate': [0.01, 0.1, 0.2]
}

# find the best hyperparameters
random_search = RandomizedSearchCV(pipeline, param_grid, n_iter=50, cv=5, n_jobs=-1, verbose=2)
random_search.fit(X_train, y_train)

y_pred = random_search.predict(X_test)
y_prob = random_search.predict_proba(X_test)[:, 1]
print(f"Best parameters found: {random_search.best_params_}")
print(f"Accuracy: {accuracy_score(y_test, y_pred)}")
print(f"ROC-AUC Score: {roc_auc_score(y_test, y_prob)}")
print(f"Classification Report:\n {classification_report(y_test, y_pred)}")

# Plot ROC curve
fpr, tpr, _ = roc_curve(y_test, y_prob)
roc_auc = auc(fpr, tpr)
plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:0.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.show()

cv_scores = cross_val_score(random_search.best_estimator_, X_selected, y, cv=5)
print(f"Cross-validation scores: {cv_scores}")
print(f"Mean cross-validation score: {cv_scores.mean()}")

# Model explainability
explainer = shap.TreeExplainer(random_search.best_estimator_.named_steps['classifier'].final_estimator_)
shap_values = explainer.shap_values(X_test)
shap.summary_plot(shap_values, X_test)

# AutoML with TPOT
tpot = TPOTClassifier(generations=5, population_size=50, verbosity=2, random_state=42)
tpot.fit(X_train, y_train)

# Evaluate TPOT model
tpot_pred = tpot.predict(X_test)
print(f"TPOT Accuracy: {accuracy_score(y_test, tpot_pred)}")
print(f"TPOT Classification Report:\n {classification_report(y_test, tpot_pred)}")
